In [6]:
import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from random import randrange
import cv2 as cv

from tensorflow import keras
from keras.layers import *
from keras.models import Sequential
from keras import optimizers


Using TensorFlow backend.


In [ ]:
DATA_FOLDER='/home/fra/Scrivania/Deepfake-Detection/dataset/kaggle'

DATA_RAW= 'raw_data'
TEST_FOLDER='test_video'
#input folder kaggle
ORIGINAL_FOLDER='original'
MANIPULATED_FOLDER='fake'

#output folder preprocess google
IMG_ORIGINAL_FOLDER  ='/home/fra/Scrivania/Deepfake-Detection/dataset/kaggle/preprocess-output/real_png'
IMG_MANIPULATED_FOLDER='/home/fra/Scrivania/Deepfake-Detection/dataset/kaggle/preprocess-output/fake_png'
IMG_TEST_FOLDER='/home/fra/Scrivania/Deepfake-Detection/dataset/kaggle/preprocess-output/test_png'

In [ ]:
#load  video

raw_data_folder=os.path.join(DATA_FOLDER, DATA_RAW)

root_folder_video_original=os.path.join(DATA_FOLDER, ORIGINAL_FOLDER)
root_folder_video_manipulated=os.path.join(DATA_FOLDER, MANIPULATED_FOLDER)
root_folder_test=os.path.join(DATA_FOLDER, TEST_FOLDER)

list_dir_original    = os.listdir(root_folder_video_original)
list_dir_manipulated = os.listdir(root_folder_video_manipulated)

train_list = list(os.listdir(raw_data_folder))
test_list=os.listdir(root_folder_test)

print(f"Train samples: {len(train_list) }")
print(f"Test samples: {len(os.listdir(root_folder_test))}")

In [ ]:
json_file = [file for file in train_list if  file.endswith('json')][0]
print(f"JSON TRAIN file: {json_file}")


json_test_file=[file for file in test_list if  file.endswith('json')][0]
print(f"JSON TEST file: {json_file}")


storage = np.array([file for file in train_list if  file.endswith('mp4')])
len(storage)

In [ ]:
def get_meta_from_json(path):
    df = pd.read_json(os.path.join(DATA_FOLDER, path, json_file))
    df = df.T
    return df

meta_train_df = get_meta_from_json(DATA_RAW)
meta_train_df.head()

In [ ]:
#Let's dividing Real video and Fake video

for video in storage:
    if meta_train_df.loc[video].label=='REAL':
        shutil.move(os.path.join(raw_data_folder,video),
                    '/home/fra/Scrivania/Deepfake-Detection/dataset/kaggle/original/'+video)    
    else:       
        shutil.move(os.path.join(raw_data_folder,video),
                    '/home/fra/Scrivania/Deepfake-Detection/dataset/kaggle/fake/'+video)


In [ ]:
storage_fake = np.array([file for file in list_dir_original if  file.endswith('mp4')])
storage_real=np.array([file for file in list_dir_manipulated if  file.endswith('mp4')])

print( len(storage_fake),len(storage_real), len(storage_fake)+len(storage_real) )

In [ ]:
FRAME_STRIDE=5   #number of frame discharged before saving 1 frame


def save_all_frame_from_video(video_path, img_dir_path):
    currentFrame = 0
    name_video_file=video_path.split('/')[-1]
    output_dir_path=os.path.join(img_dir_path,name_video_file)
    vidcap = cv.VideoCapture(video_path)
    hasFrames, frame = vidcap.read()
    frame_stride_count=0
    
    # For saving the frame in a folder
    try:
        if not os.path.exists(output_dir_path) :
            os.makedirs(output_dir_path)
            print("Directory '% s' created" % { output_dir_path }) 

            while hasFrames:
                if frame_stride_count==FRAME_STRIDE:
                    name = output_dir_path+'/' + str(currentFrame) + '.jpg'
                    cv.imwrite(name, frame)     # save frame as JPEG file      
                    hasFrames,frame = vidcap.read()
                    currentFrame += 1
                    frame_stride_count=1
                else:
                    hasFrames,frame = vidcap.read()
                    currentFrame += 1
                    frame_stride_count +=1
    except OSError:
        print (f"Error: Directory not created: { output_dir_path }")
        raise 
    
    vidcap.release()
 

In [ ]:
#                                  Uncomment to campionize the video

for video in list_dir_original:
    save_all_frame_from_video(
            os.path.join(root_folder_video_original, video), IMG_ORIGINAL_FOLDER)

for video in list_dir_manipulated:
    save_all_frame_from_video(
            os.path.join(root_folder_video_manipulated, video), IMG_MANIPULATED_FOLDER)
    
for video in test_list:
    save_all_frame_from_video(
            os.path.join(root_folder_test, video), IMG_TEST_FOLDER)
        